## Evaluation

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [4]:
df = pd.read_csv('course_lead_scoring.csv')

In [6]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [22]:
cat_col = list(df.dtypes[df.dtypes == 'object'].index)

In [23]:
cat_col

['lead_source', 'industry', 'employment_status', 'location']

In [24]:
num_col = df.select_dtypes(include=['number']).columns

In [25]:
num_col

Index(['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score', 'converted'],
      dtype='object')

## Data Preparation

In [21]:
# missing values:
# For caterogiral features, replace with 'NA'
# For numerical features, replace with with 0.0

for col in df.columns:
    if df[col].dtype == 'object': # categorical
        df.fillna({col: 'NA'}, inplace=True)
    else:  # numerical
        df.fillna({col: 0.0}, inplace=True)

## Setting up Validation Framework

In [17]:
# splitting dataset into 'train' and 'test' (80% - 20%)
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [18]:
# splitting the full training dataset into 'train' and 'valdation' (60% -20% -20%)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [19]:
# defining 'y' of all 3 datasets
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [20]:
#  to remove the 'y' variables from the dataset i.e. the target variables
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [28]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [29]:
categorical = ['location', 'lead_source', 'employment_status', 'industry']

## Training the model

In [26]:
dv = DictVectorizer(sparse=False)

In [36]:
# for training dataset
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,10000
,multi_class,'deprecated'


In [45]:
# for validation dataset
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

y_pred = model.predict_proba(X_val)[:, 1]

In [46]:
conversion_decision = (y_pred >= 0.5)

In [48]:
(y_val == conversion_decision).mean()

np.float64(0.8532423208191127)